In [149]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

from IPython.display import HTML

In [147]:
!pip install iPython

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install geopy

    100% |████████████████████████████████| 102kB 3.3MB/s a 0:00:01
  Running setup.py bdist_wheel for geographiclib ... done
  Stored in directory: /Users/edithiyerhernandez/Library/Caches/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from geopy import distance

In [5]:
tfdf = pd.read_csv('./Data/tfidf_df.csv')

In [6]:
tfdf.head()

,rest_cost,rest_name,rest_rating,address_only,rest_zip_code,11,115,11am,11pm,11th,...,cuisine_type_Vegetarian,cuisine_type_Vietnamese,cuisine_type_Wine Bar,rest_borough_Bronx,rest_borough_Brooklyn,rest_borough_Jersey City,rest_borough_Manhattan,rest_borough_Queens,rest_borough_Staten Island,rest_borough_Westchester
0,2.0,Mama’s Too,8.3,"2750 Broadway, New York, NY 10025",10025,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1,4.0,Omakase Room By Tatsu,7.7,"14 Christopher St, New York, NY 10014",10014,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
2,4.0,Sushi Azabu,8.5,"428 Greenwich St., New York, NY 10013",10013,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,3.0,Saint Julivert Fisherie,7.7,"264 Clinton St, New York, NY 11201",11201,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,2.0,Farida,8.0,"498 9th Ave, New York, NY 10018",10018,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [112]:
full_df = pd.read_csv('./Data/full_df_with_keywords.csv')

In [113]:
full_df.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,rest_cost,cuisine_type,rest_name,rest_rating,rest_review,address_only,rest_borough,rest_zip_code,lemmatized
0,0,0.0,0.3362,"ramen, pylos, pizza, essentially, ippudo, thai...",A typical NYC slice shop has a few basic eleme...,2.0,Pizza,Mama’s Too,8.3,A typical NYC slice shop has a few basic eleme...,"2750 Broadway, New York, NY 10025",Manhattan,10025,typical nyc slice shop basic element counter g...
1,1,3.0,0.6558,"sushi, pork, midtown, fish, bbq, style, blue, ...",Tolerance for group trips can vary widely. Som...,4.0,Japanese,Omakase Room By Tatsu,7.7,Tolerance for group trips can vary widely. Som...,"14 Christopher St, New York, NY 10014",Manhattan,10014,tolerance group trip vary widely people intern...
2,2,3.0,0.5712,"sushi, pork, midtown, fish, bbq, style, blue, ...",When the apocalypse eventually comes for New Y...,4.0,Japanese,Sushi Azabu,8.5,When the apocalypse eventually comes for New Y...,"428 Greenwich St., New York, NY 10013",Manhattan,10013,apocalypse eventually come new york city going...
3,3,2.0,0.5248,"chicken, meal, thing, dining, sandwich, burger...","When you’re young, you don’t have to think muc...",3.0,Seafood,Saint Julivert Fisherie,7.7,"When you’re young, you don’t have to think muc...","264 Clinton St, New York, NY 11201",Brooklyn,11201,young think much decision know attempt deep en...
4,4,0.0,0.3102,"ramen, pylos, pizza, essentially, ippudo, thai...","There’s a bleak, sweaty place on 34th Street w...",2.0,Russian,Farida,8.0,"There’s a bleak, sweaty place on 34th Street w...","498 9th Ave, New York, NY 10018",Manhattan,10018,bleak sweaty place 34th street adult run 6 yar...


In [134]:
full_df.sample()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,rest_cost,cuisine_type,rest_name,rest_rating,rest_review,address_only,rest_borough,rest_zip_code,lemmatized,lat_long,lat,long
758,758,3.0,0.1302,"sushi, pork, midtown, fish, bbq, style, blue, ...",If you were to draft up a checklist of all the...,2.0,American,Lodge,4.5,If you were to draft up a checklist of all the...,"318 Grand St., Brooklyn, NY 11211",Brooklyn,11211,draft checklist essential hipster restaurant l...,"(40.7125786, -73.9567395)",40.7126,-73.9567


In [131]:
import geopy

In [40]:
geopy.geocoders.options.default_user_agent = 'bubs'
geopy.geocoders.options.default_timeout = 7

In [41]:
from geopy.geocoders import Nominatim

In [42]:
geolocator = Nominatim(user_agent='Hello')

In [12]:
location = geolocator.geocode("428 Greenwich St., New York, NY 10013")

In [14]:
location[1]

(40.72241805, -74.0099711019111)

In [15]:
location

Location(428, Greenwich Street, TriBeCa, Manhattan, Manhattan Community Board 1, New York County, NYC, New York, 10013, USA, (40.72241805, -74.0099711019111, 0.0))

In [125]:
location[0].split(',')[2]

' TriBeCa'

In [138]:
len(geolocator.geocode('498 9th Ave, New York, NY 10018')[0].split(','))

10

In [139]:
len(geolocator.geocode('318 Grand St., Brooklyn, NY 11211')[0].split(','))

9

In [29]:
geolocator.geocode(full_df['address_only'][1])[1]

(40.7338779, -74.0004371)

In [32]:
type(geolocator.geocode(full_df.loc[1, 'address_only'])[1][1])

float

In [51]:
#create lat and long list
lat_long = []


In [53]:
for i in full_df['address_only']:
    try:
        lat_long.append(geolocator.geocode(i)[1])
    except:
        lat_long.append('none')

In [54]:
len(lat_long)

824

In [114]:
full_df['lat_long'] = lat_long

In [115]:
full_df.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,rest_cost,cuisine_type,rest_name,rest_rating,rest_review,address_only,rest_borough,rest_zip_code,lemmatized,lat_long
0,0,0.0,0.3362,"ramen, pylos, pizza, essentially, ippudo, thai...",A typical NYC slice shop has a few basic eleme...,2.0,Pizza,Mama’s Too,8.3,A typical NYC slice shop has a few basic eleme...,"2750 Broadway, New York, NY 10025",Manhattan,10025,typical nyc slice shop basic element counter g...,"(40.8008322, -73.9676555)"
1,1,3.0,0.6558,"sushi, pork, midtown, fish, bbq, style, blue, ...",Tolerance for group trips can vary widely. Som...,4.0,Japanese,Omakase Room By Tatsu,7.7,Tolerance for group trips can vary widely. Som...,"14 Christopher St, New York, NY 10014",Manhattan,10014,tolerance group trip vary widely people intern...,"(40.7338779, -74.0004371)"
2,2,3.0,0.5712,"sushi, pork, midtown, fish, bbq, style, blue, ...",When the apocalypse eventually comes for New Y...,4.0,Japanese,Sushi Azabu,8.5,When the apocalypse eventually comes for New Y...,"428 Greenwich St., New York, NY 10013",Manhattan,10013,apocalypse eventually come new york city going...,"(40.72241805, -74.0099711019111)"
3,3,2.0,0.5248,"chicken, meal, thing, dining, sandwich, burger...","When you’re young, you don’t have to think muc...",3.0,Seafood,Saint Julivert Fisherie,7.7,"When you’re young, you don’t have to think muc...","264 Clinton St, New York, NY 11201",Brooklyn,11201,young think much decision know attempt deep en...,"(40.688027, -73.995544)"
4,4,0.0,0.3102,"ramen, pylos, pizza, essentially, ippudo, thai...","There’s a bleak, sweaty place on 34th Street w...",2.0,Russian,Farida,8.0,"There’s a bleak, sweaty place on 34th Street w...","498 9th Ave, New York, NY 10018",Manhattan,10018,bleak sweaty place 34th street adult run 6 yar...,"(40.75561545, -73.9942678129495)"


In [208]:
full_df[full_df['lat_long'] == 'none']

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,rest_cost,cuisine_type,rest_name,rest_rating,rest_review,address_only,rest_borough,rest_zip_code,lemmatized,lat_long,lat,long
83,83,2.0,0.4739,"chicken, meal, thing, dining, sandwich, burger...","At Bed Bath & Beyond, you can return a waffle ...",2.0,Wine Bar,Have & Meyer,7.8,"At Bed Bath & Beyond, you can return a waffle ...","103 Havermeyer St, Brooklyn, NY 11211",Brooklyn,11211,bed bath beyond return waffle iron bought two ...,none,n,o
268,268,2.0,0.4190,"chicken, meal, thing, dining, sandwich, burger...",Every once in a while you stumble into a resta...,3.0,American,Prospect,8.4,Every once in a while you stumble into a resta...,"773 Fulton St., Brookyln, NY 11217",Brooklyn,11217,every stumble restaurant convenience expecting...,none,n,o
276,276,2.0,0.5540,"chicken, meal, thing, dining, sandwich, burger...",There’s a lot of cool sh*t that goes on in K-T...,2.0,Korean,Turntable Chicken Jazz,7.5,There’s a lot of cool sh*t that goes on in K-T...,"314 5th Ave, 2nd Floor, New York, NY 10001",Manhattan,10001,lot cool sh go k town day need know happens si...,none,n,o
294,294,2.0,0.2777,"chicken, meal, thing, dining, sandwich, burger...","If you’ve spent any time in London lately, you...",4.0,American,The Clocktower,8.2,"If you’ve spent any time in London lately, you...","5 Madison Ave., 2nd Floor, New York, NY 10010",Manhattan,10010,spent time london lately probably noticed some...,none,n,o
335,335,0.0,0.3304,"ramen, pylos, pizza, essentially, ippudo, thai...",I made a pact with my arteries the other day. ...,1.0,Bar Food,The Commodore,7.9,I made a pact with my arteries the other day. ...,"366 Metropolitian Ave., Brooklyn, NY 11211",Brooklyn,11211,made pact artery day basically agreed would ea...,none,n,o
385,385,3.0,0.5229,"sushi, pork, midtown, fish, bbq, style, blue, ...",Queens Comfort is basically what you'd get if ...,3.0,American,Queens Comfort,7.7,Queens Comfort is basically what you'd get if ...,"3618 30th Ave, Astoria, NY 11103, Queens, NY 1...",Queens,11103,queen comfort basically get pee wee herman ope...,none,n,o
466,466,3.0,0.0370,"sushi, pork, midtown, fish, bbq, style, blue, ...",New York City's floating restaurant situation ...,3.0,Raw Bar,Grand Banks,7.1,New York City's floating restaurant situation ...,"225 West St., Pier 25, New York, NY 10013",Manhattan,10013,new york city floating restaurant situation im...,none,n,o
511,511,1.0,0.2598,"mexican, pizza, tacos, slice, artichoke, pie, ...","I hope you NY1, Food Network, and Google emplo...",3.0,Seafood,Cull & Pistol,7.2,"I hope you NY1, Food Network, and Google emplo...","Chelsea Market, 75 9th Ave., New York, NY 10011",Manhattan,10011,hope ny1 food network google employee know goo...,none,n,o
591,591,3.0,0.5295,"sushi, pork, midtown, fish, bbq, style, blue, ...",It's not often you get to dine in a restaurant...,3.0,Korean,Gaonnuri,6.8,It's not often you get to dine in a restaurant...,"1250 Broadway, Penthouse, 39th Floor, New York...",Manhattan,10001,often get dine restaurant top floor skyscraper...,none,n,o
657,657,0.0,0.1901,"ramen, pylos, pizza, essentially, ippudo, thai...",It’s painful going back and reading a lot of t...,3.0,American,Freemans,8.2,It’s painful going back and reading a lot of t...,"End of Freeman Alley, New York, NY 10002",Manhattan,10002,painful going back reading lot review wrote st...,none,n,o


It turns out there are 13 observations with 'none' as their latitude and longitude - we can look these up manually.

In [221]:
null_dict = {full_df['rest_name'][i]:full_df['address_only'][i] for i in full_df[full_df['lat_long'] == 'none'].index}

In [222]:
null_dict

{'Have & Meyer': '103 Havermeyer St, Brooklyn, NY 11211',
 'Prospect': '773 Fulton St., Brookyln, NY 11217',
 'Turntable Chicken Jazz': '314 5th Ave, 2nd Floor, New York, NY 10001',
 'The Clocktower': '5 Madison Ave., 2nd Floor, New York, NY 10010',
 'The Commodore': '366 Metropolitian Ave., Brooklyn, NY 11211',
 'Queens Comfort': '3618 30th Ave, Astoria, NY 11103, Queens, NY 11103',
 'Grand Banks': '225 West St., Pier 25, New York, NY 10013',
 'Cull & Pistol': 'Chelsea Market, 75 9th Ave., New York, NY 10011',
 'Gaonnuri': '1250 Broadway, Penthouse, 39th Floor, New York, NY 10001',
 'Freemans': 'End of Freeman Alley, New York, NY 10002',
 'Yakitori Totto': '251 W. 55th St. 2nd Floor, New York, NY 10019',
 'Blue Hill at Stone Barns': '630 Bedford Rd., Pocantico Hills, NY 10591',
 'Bar Boulud': '1900 Broadway #2, New York, NY 10023'}

In [262]:
fill_null_dict = {'Have & Meyer': [40.713187, -73.956312], 
                  'Prospect': [40.686188, -73.973812],
                  'Turntable Chicken Jazz': [40.747062, -73.985938],
                  'The Clocktower': [40.741187, -73.987437],
                  'The Commodore': [40.713937, -73.955812],
                  'Queens Comfort':[40.764562, -73.916813],
                  'Grand Banks' : [40.720312, -74.014313],
                  'Cull & Pistol': [40.742562, -74.006062],
                  'Gaonnuri': [40.747687, -73.987938],
                  'Freemans': [40.721938, -73.992438],
                  'Yakitori Totto': [40.765312, -73.983187],
                  'Blue Hill at Stone Barns': [41.103062, -73.829063],
                  'Bar Boulud': [40.771563, -73.981437]}

In [265]:
fill_null_dict['Have & Meyer'][0]

40.713187

I was running into issues replacing what was a string with a tuple in the dataframe, so I worked around this challenge by just adding the latitudes to the latitude column and the longitude column.

In [266]:
for k in fill_null_dict:
    for i, r in enumerate(full_df['rest_name']):
        if k == r:
            full_df.loc[i, 'lat'] = fill_null_dict[k][0]
            full_df.loc[i, 'long'] = fill_null_dict[k][1]
            #print(fill_null_dict[k], type(fill_null_dict[k]))

Check that null values have been removed

In [267]:
full_df[full_df['lat_long'] == 'none']

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,rest_cost,cuisine_type,rest_name,rest_rating,rest_review,address_only,rest_borough,rest_zip_code,lemmatized,lat_long,lat,long


In [248]:
type(full_df.loc[83,'lat_long'])

str

In [119]:
full_df[full_df['Dominant_Topic'].isnull()]

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,rest_cost,cuisine_type,rest_name,rest_rating,rest_review,address_only,rest_borough,rest_zip_code,lemmatized,lat_long,lat,long


Now we have to fill the `'lat'` and `'long'` columns that weren't filled before.

In [240]:
#full_df['lat'] = full_df['lat_long'].map(lambda x: float(x[0]))
#full_df['long'] = full_df['lat_long'].map(lambda x: float(x[1]))

In [270]:
full_df.to_csv('./Data/full_data_with_lat_long.csv', index = False)

### Look into calculating distance using GeoPy

In [61]:
from geopy import distance

In [62]:
distance.distance(full_df.loc[1, 'lat_long'], full_df.loc[44, 'lat_long']).miles

4.556226407087412

In [66]:
print(full_df.loc[1, 'rest_name'] + ': ' + full_df.loc[1, 'address_only'])
print(full_df.loc[44, 'rest_name'] + ': ' + full_df.loc[44, 'address_only'])

Omakase Room By Tatsu: 14 Christopher St, New York, NY 10014
Lowerline: 794 Washington Ave, Brooklyn, NY 11238


In [68]:
!pip install plotly

    100% |████████████████████████████████| 31.1MB 97kB/s eta 0:00:011    33% |██████████▋                     | 10.3MB 4.3MB/s eta 0:00:05
  Running setup.py bdist_wheel for plotly ... done
  Stored in directory: /Users/edithiyerhernandez/Library/Caches/pip/wheels/67/0b/29/08c7f5caed2d1ac446db982ff607b326d49bfa0bd3a67ef8c7
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /Users/edithiyerhernandez/Library/Caches/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built plotly retrying
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [97]:
import plotly.plotly
import plotly.graph_objs as go

In [105]:
plotly.tools.set_credentials_file(username="edithiyer", api_key="PJhCwkfTHRrSNJe7RnUI")

In [77]:
full_df['lat_long'][0][0]

40.8008322

In [81]:
full_df.loc[0, 'lat_long'][0]

40.8008322

In [117]:
full_df['lat'] = full_df['lat_long'].map(lambda x: x[0])

In [118]:
full_df['long'] = full_df['lat_long'].map(lambda x: x[1])

In [107]:
scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

data = [ dict(
        type = 'scattergeo',
        locationmode = 'New York City',
        lon = full_df['long'],
        lat = full_df['lat'],
        text = full_df['rest_name'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = 'red',
            colorbar=dict(
                title="NYC Restaurants"
            )
        ))]

layout = dict(
        title = 'NYC Restaurants',
        colorbar = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='nyc_restaurants' )


High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~edithiyer/0 or inside your plot.ly account where it is named 'nyc_restaurants'


## Finding distaces between two locations

In [140]:
distance.distance(full_df.loc[3, 'lat_long'], full_df.loc[800, 'lat_long']).miles

3.543899586337327

In [141]:
!pip install folium

    100% |████████████████████████████████| 92kB 13kB/s  eta 0:00:01
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [142]:
import folium

In [185]:
import mapbox_tokens

In [145]:
folium.CircleMarker(full_df.loc[33, 'lat_long'])

#### Create a base map

In [191]:
map_hooray = folium.Map(location=[40.7128, -74.0060],
                    zoom_start = 15, tiles = 'OpenStreetMap')
map_hooray

#### Create a map with markers

In [201]:
lat_1 = full_df.loc[693, 'lat']
long_1 = full_df.loc[693, 'long']
popup_1 = full_df.loc[693, 'rest_name']

lat_2 = full_df.loc[409, 'lat']
long_2 = full_df.loc[409, 'long']
popup_2 = full_df.loc[409, 'rest_name']

avg_lat = (lat_1+lat_2)/2

avg_long = (long_1 +long_2)/2

In [ ]:
folium.Map().

In [207]:
map_with_marks = folium.Map(location=[avg_lat, avg_long],
                    zoom_start = 13, tiles = 'OpenStreetMap')

folium.Marker(location = [lat_1, long_1], tooltip=folium.Tooltip(popup_1)).add_to(map_with_marks)
folium.Marker(location = [lat_2, long_2], tooltip=folium.Tooltip(popup_2)).add_to(map_with_marks)

map_with_marks

In [150]:
def display(m, height=300):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: {1}px; '
                 'border: none"></iframe>'.format(srcdoc, height))
    return embed

In [154]:
trial_map = folium.Map(location=[37.76, -122.45])

In [160]:
m = folium.Map(location=[20, 0], zoom_start=3.5)

In [161]:
print(m)

In [164]:

from IPython.display import IFrame

IFrame('http://leaflet-extras.github.io/leaflet-providers/preview/', width=900, height=750)